In [ ]:
import os
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
from datatable import dt, fread
# With python v>3.9 need to install datatable from source for the time being
# pip install git+https://github.com/h2oai/datatable

In [ ]:
annot = pd.read_csv("PRJNA123456_extracted_condition_genotype.csv")

# Union kmers for a group of accessions and outer merge dataframes

In [ ]:
merged = pd.DataFrame(columns=['C0'])
sizes = [] # number of unique kmers following the addition of each genotype group
columns = [] # name of kmer genotype group
for geno in set(annot["genotype"].values):
    acc = annot[annot["genotype"] == geno]["run_accession"].values.tolist()
    fl1, fl2 = acc[0], acc[1]
    acc = acc[2:]
    os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl1))
    os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl1))
    os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl2))
    os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl2))
    os.system('gunzip {0}/{1}.kmc_suf.gz {0}/{1}.kmc_suf.gz {0}/{2}.kmc_suf.gz {0}/{2}.kmc_suf.gz'.format('ramdisk', fl1, fl2))
    os.system('kmc_tools simple ramdisk/{0} ramdisk/{1} union ramdisk/kmer_union0'.format(fl1, fl2))
    os.system('rm {0}/{1}.kmc_suf {0}/{1}.kmc_pre {0}/{2}.kmc_suf {0}/{2}.kmc_pre'.format("ramdisk", fl1, fl2))

    # kmc_tools can't overwrite a file so have to iterate back and 
    # forth with the output file to union merge multiple kmer files
    # This also seems to be an easier solution than the internal 'complex'
    count = 1
    for fl in acc:
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl))
        os.system('gunzip ramdisk/{0}.kmc_suf.gz'.format(fl))
        os.system('gunzip ramdisk/{0}.kmc_pre.gz'.format(fl))
        prv, nxt = str((count-1)%2), str(count%2)
        os.system('kmc_tools simple ramdisk/kmer_union{0} ramdisk/{1} union ramdisk/kmer_union{2}'.format(prv, fl, nxt))
        os.system('rm {0}/{1}.kmc_suf {0}/{1}.kmc_pre {0}/kmer_union{2}.kmc_suf {0}/kmer_union{2}.kmc_pre'.format("ramdisk", fl, prv))
        count+=1
    
    # Done union merge of group kmers. Read table 
    os.system('kmc_dump ramdisk/{0} ramdisk/kmers'.format(nxt))
    kmers = fread('ramdisk/kmer_union{0}'.format(nxt)).to_pandas()
    os.system('rm ramdisk/kmer_union{0} ramdisk/kmers'.format(nxt))
    columns.append(geno)
    merged = dd.merge(merged, kmers, how="outer", on='C0')
    merged.columns = columns # must rename to prevent multiple columns with same name
    sizes.append(merged.shape[0])
    print(len(columns), sizes[-1])

# Outer merge all samples given a list of file name accessions

In [ ]:
df = pd.read_csv("selected.csv")
acc = df["run_accession"].values.tolist()

fl1, fl2 = acc[0], acc[1]
acc = acc[2:]
os.system('cp {0}.kmc_suf.gz ramdisk/'.format(fl1))
os.system('cp {0}.kmc_pre.gz ramdisk/'.format(fl1))
os.system('cp {0}.kmc_suf.gz ramdisk/'.format(fl2))
os.system('cp {0}.kmc_pre.gz ramdisk/'.format(fl2))
os.system('gunzip ramdisk/{0}.kmc_suf.gz ramdisk/{0}.kmc_pre.gz'.format(fl1))
os.system('gunzip ramdisk/{0}.kmc_suf.gz ramdisk/{0}.kmc_pre.gz'.format(fl2))
os.system('kmc_dump ramdisk/{0} ramdisk/count'.format(fl1))
os.system('kmc_dump ramdisk/{0} ramdisk/count2'.format(fl2))
os.system('rm ramdisk/{0}.kmc_suf ramdisk/{0}.kmc_pre'.format(fl1))
os.system('rm ramdisk/{0}.kmc_suf ramdisk/{0}.kmc_pre'.format(fl2))

c1 = fread('ramdisk/count').to_pandas()
c2 = fread('ramdisk/count2').to_pandas()
merged = dd.merge(c1, c2, how="outer", on='C0')
os.system('rm ramdisk/count ramdisk/count2')

columns = ["C0",fl1, fl2]
sizes = [min([c1.shape[0], c2.shape[0]]), merged.shape[0]]

for fl in acc:
    os.system('cp {0}.kmc_suf.gz ramdisk/'.format(fl))
    os.system('cp {0}.kmc_pre.gz ramdisk/'.format(fl))
    os.system('gunzip ramdisk/{0}.kmc_suf.gz'.format(fl))
    os.system('gunzip ramdisk/{0}.kmc_pre.gz'.format(fl))
    os.system('kmc_dump ramdisk/{0} ramdisk/count'.format(fl))
    os.system('rm ramdisk/{0}.kmc_suf ramdisk/{0}.kmc_pre'.format(fl))
    c1 = fread('ramdisk/count').to_pandas()
    os.system('rm ramdisk/count'.format(fl))
    columns.append(fl)
    merged = dd.merge(merged, c1, how="outer", on='C0')
    merged.columns = columns
    sizes.append(merged.shape[0])
    print(len(columns), merged.shape[0])

In [ ]:
mini_merged = merged.iloc[:10000,:].copy()
mini_merged = mini_merged.set_index("C0")
mini_merged = mini_merged.where(mini_merged < 100, other=0)
mini_merged = mini_merged.where(mini_merged == 0, other=1)
mini_merged

In [ ]:
tmp = mini_merged[["SRR5911856", "SRR5911781"]].sum(axis=1)
tmp.where(tmp == 0, 1)

In [ ]:
sns.lineplot(x=[i for i in range(len(sizes))], y=sizes)